In [1]:
from io import StringIO
import requests
import json
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import minmax_scale

In [2]:
# The code was removed by DSX for sharing.

In [3]:
df_cleaned_materials = pd.read_csv(get_object_storage_file_with_credentials_a860223228054eadb78455bad5266913('ITTProject', 'cleaned_materials.csv'), index_col="Unnamed: 0")
df_cleaned_prove = pd.read_csv(get_object_storage_file_with_credentials_a860223228054eadb78455bad5266913('ITTProject', 'cleaned_prove.csv'), index_col="Unnamed: 0")

In [4]:
df_cleaned_materials.head()

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
8951,0.042246,-0.005937,0.027117,-0.001903,0.013791,-0.022353,0.063907,-0.023371,0.004564,-0.012443,...,0.000227,0.000112,0.000044,0.000041,-0.000015,-7.860875e-07,-0.000057,-0.000066,0.000062,0.000020
8997,0.027176,0.087921,-0.077450,0.032441,0.005934,0.029765,0.009249,0.004650,-0.003320,0.016842,...,-0.008613,-0.012683,0.004724,0.008423,0.006751,-1.920840e-03,-0.006889,-0.004928,-0.010872,-0.005499
9024,0.042246,-0.005937,0.027117,-0.001903,0.013791,-0.022353,0.063907,-0.023371,0.004564,-0.012443,...,0.000227,0.000112,0.000044,0.000041,-0.000015,-7.860875e-07,-0.000057,-0.000066,0.000062,0.000020
9043,0.052750,0.002831,0.037146,-0.011094,0.035755,-0.066244,0.203717,-0.099144,0.056779,-0.056352,...,-0.004587,-0.000884,-0.000036,-0.000659,-0.000442,-4.274769e-04,0.000617,0.000255,-0.000725,0.002245
9313,0.042074,-0.005888,0.026843,-0.001879,0.013626,-0.022069,0.063027,-0.023009,0.004487,-0.012187,...,-0.001677,-0.000871,-0.000362,-0.000424,0.000138,2.836292e-06,0.000862,0.001160,-0.001382,-0.000705


In [5]:
df_cleaned_prove.head()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,CodImpianto_13.0,CodImpianto_14.0,CodImpianto_15.0,Disco_nuovo_0,Disco_nuovo_1,Disco_256X22,Disco_256x22,Disco_276x24,Disco_280X22,Disco_280x22
9482,9482,6.7,164,166,167,753,768,54,72.5,190,...,0,0,0,1,0,0,0,0,0,1
9799,9799,9.7,169,194,184,740,742,54,65.0,195,...,0,0,0,1,0,0,0,0,1,0
16136,16136,8.9,196,196,194,768,743,54,65.0,195,...,0,0,0,1,0,1,0,0,0,0
19269,19269,7.9,194,194,190,0,0,54,65.0,195,...,0,0,0,1,0,1,0,0,0,0
19269,19269,8.8,190,213,195,0,0,54,72.5,190,...,0,0,0,1,0,0,0,0,0,1


In [6]:
print "Materials dimensionality: " + str(df_cleaned_materials.shape)
print "Prove dimensionality: " + str(df_cleaned_prove.shape)

Materials dimensionality: (482, 193)
Prove dimensionality: (653, 42)


In [7]:
df_merged = df_cleaned_prove.join(df_cleaned_materials, how = 'right')
df_merged = df_merged.reset_index(drop=True)
for col in df_merged.columns:
    df_merged[col] = df_merged[col].fillna(df_merged[col].median())
print df_merged.shape

(651, 235)


In [8]:
df_merged.head()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
0,8951,15.2,165,182,182,810,813,54,65,195,...,0.000227,0.000112,0.000044,0.000041,-0.000015,-7.860875e-07,-0.000057,-0.000066,0.000062,0.000020
1,8997,13.3,177,181,203,738,729,54,65,195,...,-0.008613,-0.012683,0.004724,0.008423,0.006751,-1.920840e-03,-0.006889,-0.004928,-0.010872,-0.005499
2,9024,7.5,235,125,136,727,731,54,65,195,...,0.000227,0.000112,0.000044,0.000041,-0.000015,-7.860875e-07,-0.000057,-0.000066,0.000062,0.000020
3,9024,7.8,237,137,102,701,674,54,65,195,...,0.000227,0.000112,0.000044,0.000041,-0.000015,-7.860875e-07,-0.000057,-0.000066,0.000062,0.000020
4,9043,10.9,180,196,201,820,805,54,65,195,...,-0.004587,-0.000884,-0.000036,-0.000659,-0.000442,-4.274769e-04,0.000617,0.000255,-0.000725,0.002245


In [9]:
df_merged.describe()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
count,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,...,6.510000e+02,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000
mean,20697.623656,13.223810,198.897081,175.374808,175.364055,207.476190,206.983103,54.110599,66.614439,191.015361,...,-3.414743e-04,-0.000225,-0.000141,-0.000045,-0.000052,0.000082,0.000056,0.000164,0.000089,0.000078
std,6260.409757,8.007487,25.994618,43.729211,44.148712,327.508577,326.780818,0.565736,3.959744,5.440001,...,6.827305e-03,0.006439,0.006349,0.006239,0.006450,0.006100,0.006369,0.006268,0.006603,0.006504
min,8951.000000,5.600000,146.000000,37.000000,36.000000,0.000000,0.000000,54.000000,45.000000,169.000000,...,-2.970575e-02,-0.037881,-0.031018,-0.062788,-0.049501,-0.121384,-0.029272,-0.076431,-0.028435,-0.026570
25%,15879.500000,8.600000,180.000000,147.000000,146.000000,0.000000,0.000000,54.000000,65.000000,185.000000,...,-2.998759e-03,-0.002843,-0.002174,-0.002547,-0.001910,-0.000886,-0.002337,-0.001556,-0.002436,-0.002884
50%,23356.000000,11.000000,194.000000,173.000000,173.000000,0.000000,0.000000,54.000000,65.000000,192.000000,...,2.249526e-07,-0.000384,-0.000112,0.000132,0.000121,-0.000055,-0.000032,-0.000036,0.000058,0.000141
75%,25291.500000,14.650000,213.000000,198.000000,197.000000,622.000000,624.500000,54.000000,65.000000,195.000000,...,2.474705e-03,0.002008,0.001728,0.002424,0.002147,0.000701,0.002616,0.001984,0.002574,0.002425
max,29526.000000,101.800000,305.000000,481.000000,489.000000,930.000000,867.000000,57.000000,72.500000,215.000000,...,3.529283e-02,0.043685,0.088749,0.049042,0.052787,0.057288,0.048384,0.032491,0.063410,0.028350


In [10]:
df_merged_zscore = df_merged.apply(zscore)
df_merged_zscore.describe()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
count,6.510000e+02,6.510000e+02,6.510000e+02,6.510000e+02,6.510000e+02,651.000000,6.510000e+02,6.510000e+02,6.510000e+02,6.510000e+02,...,6.510000e+02,6.510000e+02,6.510000e+02,651.000000,6.510000e+02,6.510000e+02,6.510000e+02,6.510000e+02,651.000000,6.510000e+02
mean,-2.619512e-16,-1.200610e-16,-4.802439e-16,-1.910061e-16,2.237500e-16,0.000000,-8.731708e-17,5.621037e-15,-3.492683e-16,1.877317e-15,...,-5.457317e-18,-2.728659e-17,-1.091463e-17,0.000000,-2.455793e-17,6.821647e-18,1.091463e-17,-1.091463e-17,0.000000,1.364329e-17
std,1.000769e+00,1.000769e+00,1.000769e+00,1.000769e+00,1.000769e+00,1.000769,1.000769e+00,1.000769e+00,1.000769e+00,1.000769e+00,...,1.000769e+00,1.000769e+00,1.000769e+00,1.000769,1.000769e+00,1.000769e+00,1.000769e+00,1.000769e+00,1.000769,1.000769e+00
min,-1.877777e+00,-9.528173e-01,-2.036489e+00,-3.166790e+00,-3.159123e+00,-0.633986,-6.338875e-01,-1.956464e-01,-5.462741e+00,-4.050053e+00,...,-4.304312e+00,-5.852851e+00,-4.867083e+00,-10.064515,-7.672663e+00,-1.992879e+01,-4.608124e+00,-1.222933e+01,-4.323208,-4.100628e+00
25%,-7.702097e-01,-5.778798e-01,-7.275203e-01,-6.493743e-01,-6.656283e-01,-0.633986,-6.338875e-01,-1.956464e-01,-4.080265e-01,-1.106615e+00,...,-3.895136e-01,-4.069013e-01,-3.205295e-01,-0.401393,-2.883393e-01,-1.588198e-01,-3.759748e-01,-2.744953e-01,-0.382751,-4.558754e-01
50%,4.249595e-01,-2.779298e-01,-1.885331e-01,-5.434889e-02,-5.358872e-02,-0.633986,-6.338875e-01,-1.956464e-01,-4.080265e-01,1.811390e-01,...,5.008740e-02,-2.465970e-02,4.428131e-03,0.028328,2.678171e-02,-2.243576e-02,-1.392869e-02,-3.193785e-02,-0.004652,9.602633e-03
75%,7.343623e-01,1.782441e-01,5.429494e-01,5.177909e-01,4.904465e-01,1.266662,1.278649e+00,-1.956464e-01,-4.080265e-01,7.330335e-01,...,4.128048e-01,3.470221e-01,2.945856e-01,0.395991,3.411255e-01,1.015565e-01,4.021077e-01,2.906525e-01,0.376626,3.610749e-01
max,1.411276e+00,1.107018e+01,4.084865e+00,6.994414e+00,7.109542e+00,2.207818,2.021307e+00,5.111262e+00,1.487492e+00,4.412330e+00,...,5.223393e+00,6.824950e+00,1.401132e+01,7.873987,8.198372e+00,9.385724e+00,7.593361e+00,5.161539e+00,9.597157,4.350499e+00


In [11]:
df_merged_zscore.head()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
0,-1.877777,0.246983,-1.305007,0.151621,0.150424,1.841134,1.855932,-0.195646,-0.408027,0.733034,...,0.083378,0.052336,0.029062,0.013708,0.005727,-0.013565,-0.017857,-0.036729,-0.004144,-0.008894
1,-1.870424,0.009522,-0.843018,0.128736,0.626455,1.621123,1.598681,-0.195646,-0.408027,0.733034,...,-1.212446,-1.936280,0.766810,1.358266,1.055534,-0.328586,-1.091330,-0.812866,-1.661278,-0.858271
2,-1.866108,-0.715357,1.389929,-1.152857,-0.892310,1.587510,1.604806,-0.195646,-0.408027,0.733034,...,0.083378,0.052336,0.029062,0.013708,0.005727,-0.013565,-0.017857,-0.036729,-0.004144,-0.008894
3,-1.866108,-0.677863,1.466927,-0.878230,-1.663026,1.508062,1.430243,-0.195646,-0.408027,0.733034,...,0.083378,0.052336,0.029062,0.013708,0.005727,-0.013565,-0.017857,-0.036729,-0.004144,-0.008894
4,-1.863071,-0.290428,-0.727520,0.472020,0.581119,1.871691,1.831432,-0.195646,-0.408027,0.733034,...,-0.622297,-0.102419,0.016413,-0.098473,-0.060509,-0.083571,0.088104,0.014528,-0.123343,0.333468


In [12]:
df_merged_minmaxscale = df_merged.apply(lambda x:(x.max()-x)/(x.max()-x.min()), axis=0)
df_merged_minmaxscale.describe()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
count,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,...,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000,651.000000
mean,0.429083,0.920750,0.667314,0.688345,0.692353,0.776907,0.761265,0.963134,0.214020,0.521405,...,0.548232,0.538339,0.742188,0.438943,0.516562,0.320173,0.622331,0.296796,0.689433,0.514783
std,0.304273,0.083238,0.163488,0.098489,0.097459,0.352160,0.376910,0.188579,0.143991,0.118261,...,0.105038,0.078939,0.053011,0.055789,0.063056,0.034139,0.082020,0.057546,0.071892,0.118418
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.205808,0.905925,0.578616,0.637387,0.644592,0.331183,0.279700,1.000000,0.272727,0.434783,...,0.504905,0.510966,0.726583,0.416869,0.495068,0.316709,0.589375,0.280083,0.662377,0.472058
50%,0.299878,0.943867,0.698113,0.693694,0.697572,1.000000,1.000000,1.000000,0.272727,0.500000,...,0.542975,0.540284,0.741953,0.437364,0.514874,0.320939,0.623472,0.298632,0.689767,0.513647
75%,0.663256,0.968815,0.786164,0.752252,0.757174,1.000000,1.000000,1.000000,0.272727,0.652174,...,0.589114,0.570434,0.759166,0.461319,0.534730,0.325591,0.653145,0.312580,0.716929,0.568726
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
df_merged_minmaxscale.head()

,Material,Usura_mat_g,Durezza_Disco,Comprex_LP,Comprex_LR,Grindo_LP,Grindo_LR,PistoneCpx,Inerzia,Vmax,...,183,184,185,186,187,188,189,190,191,192
0,1.000000,0.900208,0.880503,0.673423,0.677704,0.129032,0.062284,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
1,0.997764,0.919958,0.805031,0.675676,0.631347,0.206452,0.159170,1,0.272727,0.434783,...,0.675487,0.691069,0.701569,0.363226,0.450055,0.331382,0.711773,0.343537,0.808774,0.616340
2,0.996452,0.980249,0.440252,0.801802,0.779249,0.218280,0.156863,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
3,0.996452,0.977131,0.427673,0.774775,0.854305,0.246237,0.222607,1,0.272727,0.434783,...,0.539481,0.534210,0.740648,0.438179,0.516201,0.320636,0.623794,0.298908,0.689731,0.515836
4,0.995529,0.944906,0.786164,0.641892,0.635762,0.118280,0.071511,1,0.272727,0.434783,...,0.613546,0.546417,0.741318,0.444433,0.520374,0.323024,0.615110,0.295961,0.698293,0.475325


In [15]:
print "Dataframe zscore dimensions: " + str(df_merged_zscore.shape)
print "Dataframe minmaxscale dimensions: " + str(df_merged_minmaxscale.shape)

Dataframe zscore dimensions: (651, 235)
Dataframe minmaxscale dimensions: (651, 235)
